**ATENCIÓN - NO SE VEN LOS GRÁFICOS PUESTO A QUE PLOTLY NO LOS GUARDA PERMANENTEMENTE EN EL DOCUMENTO Y HACE CORRER EL CÓDIGO CADA VEZ**

**POR ELLO, LOS GRÁFICOS NO ESTAN VISIBLES PERO SE PUEDEN CORRER SIN PROBLEMA Y DARAN LOS OUTPUTS QUE SE RELACIONAN CON LOS ANÁLISIS QUE HICIMOS**

**ADEMÁS, PARA EFECTOS DEL REPOSITORIO, OCURRIO LO MISMO, NO SE VEN OUTPUTS PORQUE PLOTLY HACIA QUE EL DOCUMENTO PESARA MAS DE LO QUE GITHUB SOPORTA, PERO EN TÉRMINOS DE CÓDIGO ESTA TODO LO NECESARIO PARA CORRER NUESTRO LAB**

<h1><center>Laboratorio 3: La desperación de Mr. Cheems 🐼</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos - Primavera 2025</strong></center>

### Cuerpo Docente:

- Profesores: Diego Cortez, Gabriel Iturra
- Auxiliares: Melanie Peña, Valentina Rojas
- Ayudantes: Nicolás Cabello, Cristopher Urbina

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Josefa Anselmo
- Nombre de alumno 2: Tamara Carrasco


### **Link de repositorio de GitHub:** [TamoJosha](https://github.com/Tamaracarrasco/TamoJosha-Lab-de-Prog-MDS-Prim2025)

## Temas a tratar
- Aplicar Pandas para obtener características de un DataFrame.
- Aplicar Pipelines y Column Transformers

## Reglas:

- **Grupos de 2 personas**
- Fecha de entrega: Entregas Martes a las 23:59.
- Instrucciones del lab el viernes a las 16:15 en formato online. Asistencia no es obligatoria, pero se recomienda **fuertemente** asistir.
- <u>Prohibidas las copias</u>. Cualquier intento de copia será debidamente penalizado con el reglamento de la escuela.
- Tienen que subir el laboratorio a u-cursos y a su repositorio de github. Labs que no estén en u-cursos no serán revisados. Recuerden que el repositorio también tiene nota.
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Pueden usar cualquier material del curso que estimen conveniente.

### Objetivos principales del laboratorio
- Comprender cómo aplicar pipelines de Scikit-Learn para generar procesos más limpios en Feature Engineering.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `numpy`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre arreglos (*o tensores*).

## Descripción del laboratorio.

### Importamos librerias utiles 😸

In [ ]:
# Libreria Core del lab.
import numpy as np
import pandas as pd
import datetime
from IPython.display import HTML

# Libreria para plotear (En colab esta desactualizado plotly)
%pip install --upgrade plotly
import plotly.express as px
import plotly.graph_objects as go

# Librerias utiles
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer

In [ ]:
# Si usted está utilizando Colabolatory le puede ser útil este código para cargar los archivos.
# try:
  #  from google.colab import drive
   # drive.mount("/content/drive")
   # path = 'Dirección donde tiene los archivos en el Drive'
#except:
 #   print('Ignorando conexión drive-colab')

# Feature engineering en datos de retail 🛍️

### 0. Cargar Dataset

<p align="center">
  <img width=300 src="https://s1.eestatic.com/2018/04/14/social/la_jungla_-_social_299733421_73842361_854x640.jpg">
</p>

Mr. Cheems, gerente de una cotizada tienda de retail en Europa, les solicita si pueden analizar los datos de algunas de sus tiendas. En una reunión, Mr Cheems le comenta que la calidad de sus datos no es muy buena, por lo que le solicita a usted que limpie su base de datos y cree nuevos atributos relevantes para el negocio.

Por ello, el área de ventas les entrega archivo llamado `online_retail_data.pickle` el cual usted decide cargar a continuación.

In [ ]:
import plotly.io as pio

In [ ]:
# Inserte su código aquí
df_retail = pd.read_pickle("online_retail_data.pickle")

df_retail.head(5)

In [ ]:
df_retail["Price"].describe()

In [ ]:
df_retail["Quantity"].describe()

In [ ]:
df_retail.info()

### 1. Función para explorar características [0.5 puntos]

<p align="center">
  <img width=300 src="https://editor.analyticsvidhya.com/uploads/47389meme.png">
</p>




Tras inspeccionar brevemente los datos proporcionados, usted decide crear una función que realice lo siguiente:
- Plotee un histograma para las variables precios y cantidad. [0.3 puntos]
- Imprima un conteo de datos nulos por variable [0.2 puntos]

**Nota**: Para generar los gráficos no es obligatorio el uso de `plotly`, pero si es altamente recomendado. Pueden encontrar más información de esta librería en este [enlace](https://plotly.com/python/).

**Respuesta:**

In [ ]:
def explore_data(df, name):
    """
    Función que retorna histogramas de las variables Price y Quantity de un dataframe que
    contiene esas columnas.

    Parámetros:
      df: DataFrame con columnas 'Price' y 'Quantity'
      name: nombre del dataframe. Debe ser un string
      yscale: modificamos la escala en el eje Y pues no se podía graficar "bien" con plotly.

    """
    # Se agrega esto porque tuve problemas en el renderizado de vscode
    pio.renderers.default = "notebook"
    
    # Imprimimos valores nulos
    print("Valores nulos por columna")
    print(df.isna().sum())

    # Histogramas con Plotly    

    fig1 = px.histogram(
        df, x="Price", nbins=120,
        title=f"Distribución de Price del dataframe {name}",
    ) # Aquí defino el plot para Prie

    fig1.show()

    fig2 = px.histogram(
        df, x="Quantity", nbins=120,
        title=f"Distribución de Quantity del dataframe {name}",
    ) # Aquí defino el plot para Quantity
  
    fig2.show()


In [ ]:
# histogramas sin el suavizado logarítmico

explore_data(df_retail, "df_retail")

# Como se puede ver, es muy difícil identificar el comportamiento de esta variable con esta escala
# así que vamos a aplicar el log en la siguiente celda logrando así ver el comportamiento, en otra
# escala, pero de este modo si funciona en el fondo.

### 2. Eliminando outliers [1.0 puntos]

<p align="center">
  <img width=300 src="https://media.licdn.com/dms/image/C5612AQGdXKCka7HumA/article-cover_image-shrink_600_2000/0/1520056407281?e=2147483647&v=beta&t=VZcfjjzjK4LxXdZkSu1KisWC0Ry8bk4tPCn3R8aYdNM">
</p>




#### 2.1 Creando la clase IQR [0.5 puntos]

Entre las falencias de los datos, Mr. Cheems le comenta que a veces los operadores no ingresan el precio correcto de los productos. Mr. Cheems le comenta que se dio cuenta de este fenómeno porque hay productos con precios exagerádamente altos o bajos. Por lo cual usted decide eliminar outliers del dataframe a traves del rango intercuartil el cual cuenta con los siguientes pasos:

1. Calcular el primer cuartil $Q1$ y el tercer cuartil $Q3$. Hint: utilice el método `quantile()`

2. Calcular el rango intercuartil (RIC): $RIC = Q3 - Q1$

3. Calcular los límites para identificar outliers:
 - Límite inferior: $~~Q1 - \lambda \cdot RIC$
 - Límite superior: $~~Q3 + \lambda \cdot RIC$

4. Eliminar outliers: Los outliers son los datos que están por debajo del límite inferior o por encima del límite superior.


Para realizar dicha tarea, usted decide crear una clase llamada `IQR()` utilizando `BaseEstimator` y `TransformerMixin` para realizar una transformación de cada una de las columnas numéricas del DataFrame utilizando `ColumnTransformer()` más tarde. Considere que lambda debe ser $\lambda$ un parámetro a definir por el usuario.

**Hint:** tome como referencia el siguiente [enlace](https://sklearn-template.readthedocs.io/en/latest/user_guide.html#transformer).

**Nota:** No modificar el método set_output de la clase IQR

**Respuesta:**

In [ ]:
class IQR(BaseEstimator, TransformerMixin):

  def __init__(self, cte=1.5):
    self.cte = cte # cte representa lambda que define el usuario
                   # por defecto es 1.5 (Ley de Tukey) y señala que ese sería el límite
                   # para considerar un punto como outlier en base a una convención "general"
                   # respecto de estos rangos pero que de todos modos uno podría definirlo
                   # si tiene nociones reales de como se mueven los rangos para considerar 
                   # los outliers.
    pass

  def fit(self, X, y=None): # Acá se aplica lo de la guía
    # hay que obtener solo columnas numéricas
    self.num_columns = X.select_dtypes(include='number').columns # lista de columnas del dataframe. 
                                                                 # Por como será el pipeline, 
                                                                 # deberian ser las columnas 
                                                                 # numéricas. Además, me aseguro
                                                                 # que solo sean numéricas.

    # No es necesario self.X = X ya que no se busca guardar el dataframe, solo calcular los límites
    self.q1 = X[self.num_columns].quantile(0.25) # Q1
    self.q3 = X[self.num_columns].quantile(0.75) # Q3
    self.ric = self.q3 - self.q1 # RIC

    self.lim_inf = self.q1 - self.ric*self.cte #lim. inferior
    self.lim_sup = self.q3 + self.ric*self.cte #lim. superior

    return self
    

  def transform(self, X): # de lo que entendí de la guía es que acá, X se transforma con lo que aprendió en fit (los límites)
                          # o sea acá se redefinen los outliers
    
    X_no_outlier = X.copy()
    
    mask = (( X_no_outlier[self.num_columns] < self.lim_inf) |
            ( X_no_outlier[self.num_columns] > self.lim_sup))

    # Enmascarar a NaN en lugar de eliminar el registro porque esto en el fondo
    # generaria que distintas variables tuvieran distinta cantidad de registros

    X_no_outlier.loc[:, self.num_columns] =  X_no_outlier[self.num_columns].mask(mask, other=np.nan)
    return  X_no_outlier

  def set_output(self,transform='default'):
    #No modificar esta función
    return self

#### 2.2 Creación del Pipeline [0.5 puntos]

Para comenzar introduciéndose en el uso de pipeline, usted decide definir un pipeline con el Transformer previamente definido. Además, usted decide visualizar cómo cambia la distribución de las variables Precio y Cantidad antes y despues de aplicar IQR. Para ello, usted aplica los siguientes pasos:

- Definir un pipeline llamado `numeric_transformations` para las variables precio y cantidad con la transformación IQR. [0.1 puntos]
- Defina un column transformer que aplique `numeric_transformations` para las variables numéricas y `passthrough` para las variables categóricas. Adicionalmente, fije el parámetro `verbose_feature_names_out` en `False`. Ver hint al final [0.1 puntos]
- Defina el dataframe `df_iqr` aplicado el column transformer a los datos proporcionados por Mr. Cheems considerando un valor de $\lambda$ que tenga un desempeño aceptable para ambas variables. [0.1 puntos]
- Usar `explore_data` en `df_retail` y en `df_iqr`.  [0.1 puntos]
- Reportar los cambios observados en la distribución de las variables. ¿Qué sucede al aumentar el valor de lambda? [0.1 puntos]


**Hint:** El transformador `passthrough` está predefinido y es una opción que puedes usar para las columnas que no deseas transformar. Al especificar 'passthrough' para una parte de tu ColumnTransformer, las columnas correspondientes pasarán a través del ColumnTransformer sin ninguna modificación. El siguiente [enlace](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) le puede ser útil.

**Nota:** Mantenga el método set_output del column transformer con la transformación `pandas` para obtener un dataframe una vez aplicado el column transformer.

**Respuesta:**

Apóyese de la siguiente estructura para su respuesta:

In [ ]:
# Lambda de 1.5 (Ley de Tukey) - mantenemos el mismo valor de cte que en la clase para efectos de comparación y porqupe es el de la ley de Tukey por defecto

# Definicion las variables que pasarán por cada pipeline
numerical_columns = ["Quantity", "Price"]
categorical_columns = ["Invoice", "StockCode" ,"Description", "InvoiceDate", "Customer ID" ,"Country"]

# Definicion del pipeline
numeric_transformations = Pipeline([("iqr_transformer", IQR(cte=1.5))]) # el pipe espera una lista con tuplas (nombre, transformador)

# ColumnTransformer
column_transformer = ColumnTransformer([('numerical', numeric_transformations, numerical_columns),
                                        ('categorical', "passthrough", categorical_columns) # dejo las categoricas con el pss porque aun no defino como las voy a tratar
                                        ],                                                   
                                        verbose_feature_names_out=False)

column_transformer.set_output(transform='pandas') # esto es para asegurarnos que el output devuelva un dataframe y no un array

# Aplicamos ColumnTransformer a los datos

df_iqr = column_transformer.fit_transform(df_retail) # .fit() devuelve objeto entrenado no el dataframe transformado
                                                    # por eso se usa fit_trasnform que si devuelve el dataframe transformado

# Gráficos
print("Dataframe original, cte=1.5")
explore_data(df_retail, "df_retail") # Plot inicial sin suavizado logarítmico

print("Dataframe tras aplicar IQR, cte=1.5")
explore_data(df_iqr, "df_iqr") # Plot IQR sin suavizado logarítmico


In [ ]:
print("Valores nulos en el dataframe inicial, cte=1.5")
df_retail.isna().sum()

In [ ]:
# Lamda de 3 

# Definicion las variables que pasarán por cada pipeline
numerical_columns = ["Quantity", "Price"]
categorical_columns = ["Invoice", "StockCode" ,"Description", "InvoiceDate", "Customer ID" ,"Country"]

# Definicion del pipeline
numeric_transformations = Pipeline([("iqr_transformer", IQR(cte=3))]) # el pipe espera una lista con tuplas (nombre, transformador)

# ColumnTransformer
column_transformer = ColumnTransformer([('numerical', numeric_transformations, numerical_columns),
                                        ('categorical', "passthrough", categorical_columns) # acá se pudo haber usado remainder si es que
                                        ],                                                    # no se especificaban las variables no numéricas 
                                        verbose_feature_names_out=False)

column_transformer.set_output(transform='pandas')

# Aplicamos ColumnTransformer a los datos

df_iqr = column_transformer.fit_transform(df_retail) # .fit() devuelve objeto entrenado no el dataframe trasnformado, por eso se usa fit_trasnform y no .fit

# Gráficos
print("Dataframe tras aplicar IQR, cte=3")
explore_data(df_iqr, "df_iqr") # Plot IQR sin suavizado logarítmico

In [ ]:
print("Valores nulos en el dataframe inicial, cte=3")
df_retail.isna().sum()

In [ ]:
# Lambda de 5 

# Definicion las variables que pasarán por cada pipeline
numerical_columns = ["Quantity", "Price"]
categorical_columns = ["Invoice", "StockCode" ,"Description", "InvoiceDate", "Customer ID" ,"Country"]

# Definicion del pipeline
numeric_transformations = Pipeline([("iqr_transformer", IQR(cte=5))]) # el pipe espera una lista con tuplas (nombre, transformador)

# ColumnTransformer
column_transformer = ColumnTransformer([('numerical', numeric_transformations, numerical_columns),
                                        ('categorical', "passthrough", categorical_columns) # acá se pudo haber usado remainder si es que
                                        ],                                                    # no se especificaban las variables no numéricas 
                                        verbose_feature_names_out=False)

column_transformer.set_output(transform='pandas')

# Aplicamos ColumnTransformer a los datos

df_iqr = column_transformer.fit_transform(df_retail) # .fit() devuelve objeto entrenado no el dataframe transformado, 
                                                        #por eso se usa fit_trasnform y no .fit

# Gráficos
print("Dataframe tras aplicar IQR, cte=5")
explore_data(df_iqr, "df_iqr") # Plot IQR sin suavizado logarítmico

In [ ]:
print("Valores nulos en el dataframe inicial, cte=5")
df_retail.isna().sum()

**RESPUESTA**

Lo que vimos es que al variar la cte (λ) entre 1.5, 3 y 5 cambia harto la distribución porque básicamente estás definiendo qué tan “tolerante” eres con los valores extremos. Mientras más grande es λ, más se amplía el rango en que se consideran datos “normales”, entonces menos observaciones se marcan como outliers y, en consecuencia, menos datos terminan como nulos tras aplicar la regla del IQR. Ejemplo concreto con λ = 1.5, en Quantity se generan más de 33 mil nulos, mientras que con λ = 3 bajan a 26 mil y con λ = 5 caen a 18 mil aprox. Lo mismo en Price, donde pasamos de 40 mil nulos (λ = 1.5) a 18 mil (λ = 3) y 11 mil (λ = 5). O sea, a medida que sube λ, el set se “relaja” y se eliminan menos observaciones.

Entonces, un λ chico (ej. 1.5) es más estricto y limpia fuerte las colas, un λ grande (ej. 5) deja pasar muchos más valores como “válidos” aunque estén alejados de la mediana. Lo clave es que la elección de λ no sea al azar, sino que dependa del conocimiento de la variable y del contexto de negocio/tema. Por algo Tukey propuso la regla de 1.5 como referencia general (“la pulgada”), pero igual puede ajustarse si tenemos fundamentos sólidos para hacerlo.

Si miramos los gráficos, el contraste queda clarísimo pues en el dataset original (df_retail) tanto Price como Quantity aparecen ultra sesgados a la derecha, con un cerro de observaciones acumuladas en valores muy bajos y colas larguísimas que llegan hasta 10k–18k. Esto hace que la mayoría de los bins se aplasten y casi no se aprecien los patrones internos. Cuando aplicamos IQR con λ = 1.5, esas colas se recortan fuerte y las escalas del eje x bajan harto, dejando las distribuciones mucho más “visibles” en el rango 0–8 en Price y 0–25 en Quantity. Con λ = 3 las colas vuelven a alargarse un poco y se empieza a ver más dispersión, y con λ = 5 ya casi se parece a la forma original, porque entran nuevamente valores que antes se habían tratado como outliers. Entonces, a mayor λ, aumenta el rango visible de los ejes en los gráficos porque se considera “normal” una mayor cantidad de valores extremos, mientras que con λ más chico el corte es más duro y se pierde parte de la cola.

### 3. Agregando un imputer al pipeline [1.0 puntos]



<p align="center">
  <img width=300 src="https://media.makeameme.org/created/hmm-there-is.jpg">
</p>

Para continuar con la limpieza del dataframe usted decide imputar los datos nulos de las variables numéricas, para lo cual decide realizar las siguientes tareas:

1. Crear un pipeline para variables categóricas llamado `categoric_transformations` con un paso llamado `mode_imputer`, en el cual se imputen los datos faltantes por la categoría más frecuente.
2. Agregar al pipeline `numeric_transformations` un paso llamado `mean_imputer`, en el cual se imputen los datos por la media usando [SimpleImputer](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html) [0.1 puntos]
3. Crear y aplicar un `ColumnTransformer` actualizado con los pipelines `categoric_transformations` y `numeric_transformations` a `df_retail`, creando un dataframe llamado `df_mean_imputer`. [0.1 puntos]
4. Comparar los resultados de `explore_data` en `df_mean_imputer` y `df_iqr`. ¿Qué diferencias observa en la distribución de los datos? [0.2 puntos]
5. Cambiar el imputer de `numeric_transformations` por [KNNImputer](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html) y definir un nuevo dataframe llamado `df_knn_imputer`, aplicando el nuevo ColumnTransformer a `df_retail`. En caso de los tiempos de ejecución sean altos puede probar a reducir el parámetro `n_neighbors`. [0.1 puntos]
6. Comparar los resultados de `explore_data` en `df_knn_imputer` y `df_iqr`. ¿Qué diferencias observa en la distribución de los datos? [0.2 puntos]
7. Comparar los resultados de `explore_data` en `df_knn_imputer` y `df_mean_imputer`. ¿Cuál método de imputación es mejor? Deje el método escogido en el ColumnTransformer. [0.2 puntos]

**Nota: Fije el parámetro verbose_feature_names_out en `False` y utilice el método set_output con transformación `pandas` en cada ColumnTransformer para obtener como salida un dataframe.**

**Respuesta:**

**ESTA CELDA TARDA ENTRE 20 Y 30 MIN EN CORRER SEGÚN COMPUTADOR**

In [ ]:
from sklearn.impute import SimpleImputer, KNNImputer

# Defino los tipos de columnas
# Estos son los tipos de datos que tenemos en la base category, object, datetime64[ns] y float64 entonces definimos por separado la agrupación para que después se apliquen las imputaciones
# correspondientes

numeric_cols   = df_retail.select_dtypes(include='float64').columns.tolist()
categoric_cols = df_retail.select_dtypes(include=['object','category','datetime64[ns]']).columns.tolist()

# PARTE 1 

categoric_transformations = Pipeline(steps=[('mode_imputer', SimpleImputer(strategy='most_frequent'))]) # Aquí estamos definiendo el pipe que en el fondo imputará a las variables categóricas 
                                                                                                        # el valor más frecuente de la categoría
                                                                                                        # Si bien desde un punto de vista de "imputación" esto se podría hacer, no significa
                                                                                                        # que esta sea la forma correcta de imputar todas estas clases de datos. Estrictamente 
                                                                                                        # deberíamos analizar variable por variable e imputar según correspnda a la naturaleza 
                                                                                                        # de la misma y lo que se necesite encontrar por decir.
                                                                                                        # Acá no se explicitaba pero igual usamos SimpleImputer porque de esta forma salia bien.

# PARTE 2 
# Aquí piden agregar un paso de "mean_imputer" en el pipe de "numeric_transformations" por lo que se entiende que debemos mantener el mismo pipe que teniamos definido de antes y a este
# agregarle el paso que nos comentan. No se esta pidiendo que redefinamos o que creemos un nuevo pipe por decir, más solo agregar el paso.

numeric_transformations = Pipeline([("iqr_transformer", IQR(cte=1.5)), ("mean_imputer", SimpleImputer(strategy="mean"))]) # El segundo parentesis, por decir, es el paso que se agrega acá porque 
                                                                                                                        # el resto venia de la definición inical de este pipe de una sección previa

# PARTE 3

ct_actualizado = ColumnTransformer( # con esto estoy creando el columntransformer para aplicar en la bbdd con el nombre de ct_actualizado
    transformers=[ #aqui defino las diferentes transformaciones que voy a a plicar sobre las diferentes columnas o tipos de datos en vd
        ('categoric_transformations', categoric_transformations, categoric_cols), # se compone como del nombre de la transformacion + el pipeline + las colunas donde lo aplicare que en este caso las definimos al inicio segun el tipo de datos que son
        ('numeric_transformations',   numeric_transformations,   numeric_cols)
    ],
    verbose_feature_names_out=False, # esto que tmb aplicamos antes es como para que las columnas no queden renombradas
    remainder='passthrough' # esto es como para que no tire error/levante alerta si hay columnas en el df que no se consideren dentro de las transformaciones, entonces así los deje tan cual
).set_output(transform='pandas') # por defecto ct me devuelve un array pro yo quiero el dataframe de pandas entonces lo fuerzo a que eso pase por decri

df_mean_imputer = ct_actualizado.fit_transform(df_retail) # con esto obtengo el df transformado pero las col no necesarimente están en el mismo orden
df_mean_imputer = df_mean_imputer[df_retail.columns] # Con esto puedo volverlas al mismo orden del inicio


# PARTE 4 
# ahora voy a comparar los datos de "df_mean_imputer" y "df_iqr" según lo definido al inicio en el "explore_data"

print("PARTE 4: explore_data(df_mean_imputer) vs explore_data(df_iqr)")
explore_data(df_mean_imputer, "df_mean_imputer")
explore_data(df_iqr, "df_iqr")

# PARTE 5 

# aqui redefino el pipe (en verdad cree uno nuevo por knn) con el nuevo imputador
numeric_transformations_knn = Pipeline([("iqr_transformer", IQR(cte=1.5)), ("knn_imputer", KNNImputer(n_neighbors=2, weights="uniform", metric="nan_euclidean"))])


ct_knn = ColumnTransformer( # Defino el columntransform
    transformers=[
        ('categoric_transformations', categoric_transformations, categoric_cols), # defino los cambios sobre los tipos de varibales
        ('numeric_transformations_knn',   numeric_transformations_knn,   numeric_cols),
    ],
    verbose_feature_names_out=False, # evito que cambie el nombre de la columna
    remainder='passthrough' # aqqui para que no modifique o tire error si hay variables que no tendran modificacion
).set_output(transform='pandas') # forzar al df de pandas en lugar del array

df_knn_imputer = ct_knn.fit_transform(df_retail) # aqui lo aplico ya a df_retail
df_knn_imputer = df_knn_imputer[df_retail.columns] # reordeno las columnas nuevamente


# PARTE 6
# aca comparo también los diferentes dataframes con las transformaciones de cada uno

print("PARTE 6: explore_data(df_knn_imputer) vs explore_data(df_iqr)")
explore_data(df_knn_imputer, "df_knn_imputer")
explore_data(df_iqr, "df_iqr")

# PARTE 7
# Primero tengo que definir la media y desviaion estandar de las varibales numericas, porque este será como mi 
# baseline para comparar que imputador es mejor

ref_mean = df_retail[numeric_cols].mean()
ref_std  = df_retail[numeric_cols].std(ddof=1)

# Hago como un cálculo de "distancias simples" pero en valor absoluto (|Δ media| + |Δ std|). 
# Mientras menor, mejor por que en el fondo implicaría que esta más cerca de lo "real".
dist_mean = (ref_mean - df_mean_imputer[numeric_cols].mean()).abs().sum() \
          + (ref_std  - df_mean_imputer[numeric_cols].std(ddof=1)).abs().sum()

dist_knn  = (ref_mean - df_knn_imputer[numeric_cols].mean()).abs().sum() \
          + (ref_std  - df_knn_imputer[numeric_cols].std(ddof=1)).abs().sum()

print("PARTE 7: KNN vs MEAN (comparación cuantitativa simple)")
print(f"Distancia media+std vs original -> MEAN: {dist_mean:.6f} | KNN: {dist_knn:.6f}")

# aca comparo los resultados
print("explore_data: df_knn_imputer vs df_mean_imputer")
explore_data(df_knn_imputer, "df_knn_imputer")
explore_data(df_mean_imputer, "df_mean_imputer")

# hago un if que me deje automaticamente elegir con cual nos vamos a quedar
if dist_knn <= dist_mean:
    ct_final = ct_knn
    df_imputer_final = df_knn_imputer
    metodo_elegido = "KNNImputer"
else:
    ct_final = ct_actualizado
    df_imputer_final = df_mean_imputer
    metodo_elegido = "SimpleImputer(mean)"

print(f"Método de imputación escogido: {metodo_elegido}")

**RESPUESTAS A LAS PREGUNTAS 4, 6 Y 7**

4 --> Al comparar df_mean_imputer con df_iqr se nota que el primero queda sin nulos y mucho más “limpio” para modelar, pero al mismo tiempo sus distribuciones aparecen más apretadas en el centro y con colas cortas, porque al imputar con la media todos los huecos se van al valor central y eso reduce la dispersión. En cambio, con df_iqr quedan varios nulos, sobre todo en Quantity y Price, ya que ahí se marcan outliers según el rango intercuartílico, pero lo interesante es que en los gráficos se alcanzan a ver colas mucho más largas y rangos más amplios: en Price la escala llega hasta 16 en vez de 7.5 y en Quantity hasta 60 en vez de 25, lo que refleja que el IQR mantiene la variabilidad y la asimetría real de los datos en vez de suavizarla. En la práctica entonces, el mean_imputer da un dataset más estable y sin faltantes, aunque un poco artificial en su forma, mientras que el IQR conserva mejor la heterogeneidad del negocio pero al costo de tener que decidir después cómo tratar esos nulos. Además, al variar la constante λ lo que pasa es que con valores más chicos (ej. 1.5) se eliminan más datos y se cortan fuerte las colas, mientras que con valores grandes (ej. 5) entran más observaciones dentro del rango “válido”, se generan menos nulos y se recupera la forma con colas largas que vimos en los histogramas.

6 --> Cuando comparamos el df_knn_imputer con el df_iqr se nota que con KNN el dataset queda sin nulos y las distribuciones se ven mucho más “compactas”, porque los valores faltantes se imputan a partir de vecinos y eso hace que las formas se mantengan ordenadas y sin tantos vacíos, mientras que con IQR aparecen hartos nulos en Quantity y Price y eso se refleja en histogramas con colas más largas y rangos más amplios, por ejemplo en Price el KNN se queda más acotado hasta 7 y en el IQR se estira hasta 16, y en Quantity pasa algo similar, donde KNN mantiene el rango hasta aprox. 25 mientras que el IQR llega a más de 60; en el fondo KNN te entrega un dataset operativo y sin huecos pero suaviza bastante la dispersión real, mientras que IQR conserva la variabilidad y los extremos del negocio aunque a costa de dejar faltantes que después hay que tratar, o sea es como más fiel a la data original pero menos práctico de usar tal cual.

7 --> Comparando df_knn_imputer con df_mean_imputer nos quedamos con KNN pues ambos dejan cero nulos, pero el mean empuja los huecos al centro y te “aplana” un poco las colas (queda todo más parejito pero medio artificial), en cambio KNN rellena usando vecinos y la forma de las distribuciones se ve más parecida a la original, con la variabilidad donde corresponde; de hecho en la comparación cuantitativa que corrimos la distancia media+std vs el original dio KNN = 133.385891 vs MEAN = 134.122979, o sea KNN queda más cerca del original (poquito, pero gana), y visualmente también se nota menos sesgo al centro. Así que, por fidelidad y porque no mete tanto “promedio” donde no toca, escogemos KNN y lo dejamos como método en el ColumnTransformer. 

### 4. Creación de nuevas features [2.0 puntos]

<p align="center">
  <img width=250 src="https://miro.medium.com/max/1000/1*JtTWgAcfVTWV8OTjT47Atg.jpeg">
</p>


#### 4.1 Definicion de LRMFP [1.0 puntos]

Dado que Mr. Lepin está interesado en obtener nuevos atributos relevantes para su negocio, su equipo de expertos sugiere la construcción de variables **LRMFP**, las que se construyen en base a las siguientes definiciones:

- **Length (L)**: Intervalo de tiempo, en días, entre la primera y la última visita del cliente. Mientras mas grande sea el valor, mas fiel es el cliente.

- **Recency (R)**: Indica hace cuanto tiempo el cliente realizo su ultima compra. Notar que para este caso, mientras mas grande es el valor, menos interes posee el usuario para repetir una compra en uno de los locales. **Considere "hoy" como la fecha mas reciente del dataset**.

- **Monetary (M)**: El término "monetario" se refiere a la cantidad media de dinero gastada por cada visita del cliente durante el período de observación y refleja la contribución del cliente a los ingresos de la empresa.

- **Frequency (F)**: Se refiere al número total de visitas del cliente durante el periodo de observación. Cuanto mayor sea la frecuencia, mayor será la fidelidad del cliente.

- **Periodicity (P)**: Representa si los clientes visitan las tiendas con regularidad.

$$Periodicity(n)=std(IVT_1, ..., IVT_n)$$

Donde $IVT$ denota el tiempo entre visitas y n representa el número de valores de tiempo entre visitas de un cliente.


$$IVT_i=date\_diff(t_{i+1},t)$$

En base a las definiciones señaladas, diseñe una función que permita obtener las características **LRMFP** recibiendo un DataFrame como entrada. Para esto, no estará permitido el uso de iteradores, utilice todas las herramientas que les ofrece `pandas` para realizar esto.

Una referencia que le puede ser útil es el [documento original](https://www.researchgate.net/publication/315979555_LRFMP_model_for_customer_segmentation_in_the_grocery_retail_industry_a_case_study) en donde se propone este método.

**<u>Formato</u> del Resultado Esperado:**

| Customer ID | Length | Recency | Frequency | Monetary | Periodicity |
|------------:|-------:|--------:|----------:|---------:|------------:|
|   12346.0   |    294 |      67 |        46 |   -64.68 |        37.0 |
|   12347.0   |     37 |       3 |        71 |  1323.32 |         0.0 |
|   12349.0   |    327 |      43 |       107 |  2646.99 |        78.0 |
|   12352.0   |     16 |      11 |        18 |   343.80 |         0.0 |
|   12356.0   |     44 |      16 |        84 |  3562.25 |        12.0 |

**Respuesta:**

In [ ]:
def custom_features(dataframe_in):
    
    copia = dataframe_in.copy()

    copia = copia.sort_values(["Customer ID", "InvoiceDate"]) # se ordena porque será más útil para adelante

    # PARA LENGHT

    # Se obtienen columnas customer id, primera_visita y ultima_visita
    len = copia.groupby("Customer ID", observed=True)["InvoiceDate"].agg(primera_visita="min", ultima_visita="max").reset_index() # esto es un dataframe
   
    Lenght = (len["ultima_visita"] - len["primera_visita"]).dt.days # diferencia en días: esto no es un dataframe
    
    # PARA RECENCY
    Recency = np.abs((len["ultima_visita"] - pd.to_datetime("today")).dt.days) # valor absoluto: esto no es un dataframe

    # PARA MONETARY
    copia["dinero_gastado"] =  copia["Quantity"] * copia["Price"]
    
    Monetary = copia.groupby("Customer ID", observed=True)["dinero_gastado"].mean().round(2).fillna(0).reset_index(name="Monetary") 
    # promedio gastado por cliente: esto es un dataframe
    # acá me di cuenta que hay un producto con precio 0.0, luego la multiplicacion da nan
    
    # PARA FREQUENCY
    copia["fecha"] = copia["InvoiceDate"].dt.normalize() # convierte a midnight pero quedan los dd/mm/aaaa

    Frequency = copia.groupby("Customer ID", observed=True)["fecha"].nunique().reset_index(name="Frequency") 
    # dataframe con  columnas customer id, fecha y frequency

    # PARA PERIODICIDAD
    
    copia["IVT"] = copia.groupby("Customer ID", observed=True)["InvoiceDate"].diff().dt.days # obtenemos la diferencia de dias para cada cliente

    periodicidad = copia.groupby("Customer ID", observed=True)["IVT"].std().round(1).fillna(0).reset_index(name="Periodicity") # se aplica desviacion estandar y reseteamos index
    # periodicidad es dataframe con columnas customer id y periodicidad (periodicity)

    # si un cliente visita 1 solo dia, no hay std -> se tiene un nan
    # Si un cliente visita 2 dias, hay solo un intervalo -> no hay desviación estandar -> arroja nan
    
    dataframe_out = pd.DataFrame({
        "Customer ID": len["Customer ID"],
        "Lenght": Lenght,
        "Recency": Recency,
        "Monetary": Monetary["Monetary"],
        "Frequency": Frequency["Frequency"] ,
        "Periodicity": periodicidad["Periodicity"]

    })
    return dataframe_out

#### 4.2 Agregando las custom features [1.0 puntos]

Ahora, usted decide agregar al pipeline las nuevas variables creadas, para lo cual realiza las siguientes tareas:

1. Cree un nuevo pipeline llamado `retail_pipeline` que encapsule el ColumnTransformer y calcule las LRMFP. El primer paso del pipeline llámelo  `col_tranformer` y el segundo paso llámelo `custom_features`, incorpora las nuevas variables al dataframe. Hint: les puede ser útil investigar [este](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.FunctionTransformer.html) método. [0.1 puntos]
2. Aplicar el pipeline actualizado a los datos proporcionados por Mr. Cheems, creando un nuevo dataframe llamado `df_custom`. [0.1 puntos]
3. Explorar la distribución de las nuevas variables con `explore_data` y comentar brevemente (2-3 líneas) características de cada custom feature. [0.5 puntos]
5. Entregar un insight para el negocio en base a las nuevas variables. [0.3 puntos]

**Nota:** Recuerde fijar el parámetro `verbose_feature_names_out` en `False` e incorporar el método `set_output` para obtener una salida en formato dataframe del ColumnTransformer.

**Respuesta**

**ESTA CELDA TARDA EN CORRER ENTRE 25 A 35 MINUTOS SEGUN COMPUTADOR**

In [ ]:
custom_features_transformer = FunctionTransformer(custom_features) # uso del hint que pusieron

retail_pipe = Pipeline(steps=[
                                ("col_transformer", ct_knn), 
                                ("custom_features", custom_features_transformer)    
                            ])                                                      # se escogió el col_trans de knn pq salio como el mejor para imputar

df_custom = retail_pipe.fit_transform(df_retail) # acá, se realizan las imputaciones correspondientes con knn

In [ ]:
# Definición de una nueva funcion explore_data

def explore_custom(df, name):
    """ Función que recibe un dataframe, el cual debe contener las columnas
    "Lenght", "Recency", "Monetary", "Frequency", "Periodicity" y 
    retorna un histograma de cada variable.

    Parámetros:
    --------------
    df: dataframe a tratar.

    name: (str) Nombre del dataframe.

    yscale: (str) Modifica la escala del eje y porque a veces no
                  se puede graficar bien con plotly debido a la cantidad 
                  de los datos.


    """
    
    pio.renderers.default = "notebook"

    # Impresión de valores nulos por columna
    print("Valores nulos por columna")
    print(df.isna().sum()) # debería dar 0 porque hubo imputación en todas las variables.

    # Histogramas

    # Lenght

    fig1 = px.histogram(
        df, x = "Lenght", nbins=120,
        title=f"Distribución de Length del dataframe {name}",
    )

    fig1.show()

    # Recency

    fig2 = px.histogram(
        df, x = "Recency", nbins=120,
        title=f"Distribución de Recency del dataframe {name}",
    )

    fig2.show()

    # Monetary

    fig3 = px.histogram(
        df, x = "Monetary", nbins=120,
        title=f"Distribución de Monetary del dataframe {name}",
    )

    fig3.show()
    
    # Frequency

    fig4 = px.histogram(
        df, x = "Frequency", nbins=120,
        title=f"Distribución de Frequency del dataframe {name}",
    )

    fig4.show()

    # Periodicidad

    
    fig4 = px.histogram(
        df, x = "Periodicity", nbins=120,
        title=f"Distribución de Periodicity del dataframe {name}",
    )

    fig4.show()


In [ ]:
explore_custom(df_custom, "df_custom")

Descripción de las distribuciones de las variables LRMFP:

**Lenght**: Se ve que hay una gran concentración de los datos
en los primeros bins, lo cual significa que hay una gran cantidad
de clientes que fueron a comprar pocas veces, ya que la diferencia
en días entre la última visita y la primera visita es muy pequeña.
Sin embargo hay una gran cola larga a la derecha de la distribución,
dando cuenta que hay un gran número de clientes que fueron fieles
a la empresa durante el períodode observación, ya que hicieron compras/visitas por mucho tiempo, es decir, 
la diferencia en días entre su última visita y primera visita es
alta.

**Recency**: Se observa una distribución asimétrica sesgada a la derecha. También nos fijamos en los valores del eje x, que indica
que las fechas disponibles del dataset corresponden a un período antiguo en comparación a la fecha de hoy.
El díficil interpretar si los clientes presentarían interés en comprar en el día de hoy ya que la data es antigua y se obtiene una Recency del orden de
5* 10^3. Pero hay una gran cantidad de clientes que se concentra 
en los menores valores obtenidos de Recency: La mayoría de los clientes se concentra
en Recency $\in$ [5400, 5450].
Aproximadamente, hay un cuarto del total de clientes que fueron fieles en el período de observación.

**Monetary**: Se observa una distribución asimétrica sesgada a la derecha,
de lo que se desprende que casi la mitad de los clientes (~2000) hacen un aporte
monetario promedio de 12 a 23 unidades monetarias, siendo
el intervalo [15-17] el que posee más clientes (~700) que realizan ese aporte en 
promedio. 
Hay pocos clientes (<100) que realizan en promedio grandes aportes monetarios a la empresa (más de 40 unidades
monetarias).

**Frequency**: La mayoría de los datos están cargados a la izquierda, 
donde el rango de frecuencia [2,3] es el que más posee conteos.
Alrededor de 1514 clientes realizaron alrededor de 2 o 3 visitas en promedio.
Luego, hay ~900 clientes que visitaron solo una vez y 781 clientes que visitaron entre 4 y 5 veces.

La cantidad de clientes que visita más de 5 veces la tienda, disminuye exponencialmente.


**Periodicidad**: Hay una gran cantidad de clientes que presentan periodicidad nula.
Esto es esperable ya que hay muchos clientes que solo visitaron la 
tienda 1 vez o 2 veces , luego el IVT es 0 o 1, intervalos que no dispersión.
(y esos valores se fijaron como 0 para no remper la integridad del dataframe LRMFP).
Por otra parte, hay clientes que presentan alta periocididad; hay una gran concentración
de clientes (~800) que presentan periodicidad $\in$ [7,13]. Luego
casi un cuarto de los clientes si visitaron la tienda con regularidad.


**Insight**: Después de analizar la distribución de las nuevas variables de la
data disponible,
el negocio si presentó una clientela fiel durante el período de observación, que visita en promedio 
2 a 5 veces y que en promedio realiza un aporte importante
de ~15 unidades momenetarias, sin embargo se recomendaría 
consultar con un experto de marketing 
o de negocios para que la empresa sea más considerada con esos clientes
para aumentar su leatad y también se recomienda consultar con expertos sobre
políticas para capturar y evitar fugas de clientes que pueden ser ctalogados como ocasionales: frecuencia baja, aporte monetario moderado a la empresa y
periodicidad nula.

Ahora otro punto a considerar es que si se desean implementar ciertas políticas con respecto a los clientes, debería usarse data más actual
para tomar decisiones informadas, ya que los contextos temporales de las compras pueden ser distintos.


### 5. MinMax Scaler [1.0 puntos]

<p align="center">
  <img width=300 src="https://i.imgflip.com/1fsprn.jpg">
</p>


#### 5.1 Definición del Column Transformer [0.5 puntos]

Construya una clase llamada `MinMax()` para realizar una transformación de cada una de las columnas de un DataFrame utilizando `ColumnTransformer()`. Recuerde  usar `BaseEstimator` y `TransformerMixin`.


 Para esto considere que Min-Max escaler queda dada por la ecuación:

$$MinMax = \dfrac{x-min(x)}{max(x) - min(x)}$$


Consulte el siguiente [link](https://sklearn-template.readthedocs.io/en/latest/user_guide.html#transformer) si tiene dudas sobre la creación de custom transformers.

**Respuesta:**

In [ ]:
class MinMax(BaseEstimator, TransformerMixin):
    """
    Escala cada columna a [0, 1] con la fórmula:
        MinMax = (X - min_col) / (max_col - min_col)
    - Deja NaN sin tocar (np.nanmin/np.nanmax ignoran NaN).
    - Si max == min (columna constante), uso denominador 1 para evitar división por cero,
      por lo que la columna resultará en ceros.
    """

    def fit(self, X):
        # Guardo si me pasaron un DataFrame para poder devolver el mismo tipo en transform()
        self._is_df_ = isinstance(X, pd.DataFrame)
        if self._is_df_:
            # Si es DataFrame, recuerdo columnas e índice
            self._cols_  = X.columns
            self._index_ = X.index
            # Convierto a ndarray float para calcular min/max/rango de forma homogénea
            Xv = X.values.astype(float)
        else:
            # Si no es DataFrame, aseguro que sea un ndarray float
            Xv = np.asarray(X, dtype=float)

        # Calculo mínimos por columna ignorando NaN
        self.data_min_ = np.nanmin(Xv, axis=0)
        # Calculo máximos por columna ignorando NaN
        self.data_max_ = np.nanmax(Xv, axis=0)
        # Rango por columna (max - min)
        self.range_    = self.data_max_ - self.data_min_

        # Evito división por cero así que donde el rango sea 0, lo reemplazo por 1 (así toda la columna constante queda en 0 tras escalar)
        self.range_safe_ = np.where(self.range_ == 0.0, 1.0, self.range_)

        # Devuelvo self ajustado
        return self

    def transform(self, X):

        # Detecto si ahora me pasan DataFrame para preservar nombres/índice
        is_df = isinstance(X, pd.DataFrame)
        if is_df:
            # Guardo columnas e índice actuales
            cols = X.columns
            idx  = X.index
            # Trabajo internamente con ndarray float
            Xv = X.values.astype(float)
        else:
            # Si me pasan ndarray, intento recuperar columnas guardadas en el fit
            cols = getattr(self, "_cols_", None)
            idx  = None
            # Me aseguro de tener un ndarray float
            Xv = np.asarray(X, dtype=float)

        # Aplico min-max por columnas: (X - min) / range_safe_
        X_scaled = (Xv - self.data_min_) / self.range_safe_

        # Si el input fue DataFrame, devuelvo un DataFrame con mismos nombres/índice
        if is_df:
            return pd.DataFrame(X_scaled, columns=cols, index=idx)
        else:
            # Si fue ndarray, devuelvo ndarray
            return X_scaled

    def set_output(self, transform='default'):
        # No modifico este método (según enunciado); retorno self para encadenar
        return self

#### 5.2 Incorporando MinMax al pipeline [0.5 puntos]

Ahora, usted decide agregar el escalamiento al pipeline, para lo que decide seguir los siguientes pasos:

- Agregar el paso `minmax` al pipeline `numeric_transformations`, haciendo uso de la clase creada. [0.1 puntos]
- Defina el dataframe `df_minmax` aplicando el ColumnTransformer actualizado a los datos proporcionados por Mr. Cheems. [0.1 puntos]
- Usar `explore_data` en `df_retail` y en `df_minmax`. [0.1 puntos]
- Reportar los cambios observados en la distribución de las variables.  [0.2 puntos]

**Nota:** Recuerde fijar el parámetro `verbose_feature_names_out` en `False` e incorporar el método `set_output` para obtener una salida en formato dataframe del ColumnTransformer.

**Respuesta:**

**ESTA CELDA TARDA ENTRE 25 A 30 MINUTOS EN CORRER SEGÚN COMPUTADOR**

In [ ]:
# Considero pipeline y columntransformer que ya salio antes como el mejor -> el de knnimputer

# redefino el pipeline numérico para agregar el paso MinMax
numeric_transformations_knn = Pipeline(steps=[("iqr_transformer", IQR(cte=1.5)), ("knn_imputer", KNNImputer(n_neighbors=2, weights="uniform", metric="nan_euclidean")), ("minmax", MinMax()),]) # aquí entonces sumo los pasos de "recorte" de outputs, 
                                                                                                                                                                                                # imputo con knn y suavizo a [0 y 1] con minmax este ultimo paso

# redefino ColumnTransformer basado en mi ct_knn anterior, pero ahora con el pipeline que incluye MinMax

ct_knn_minmax = ColumnTransformer(
    transformers=[
        ('categoric_transformations', categoric_transformations, categoric_cols),  # categóricas igual que antes
        ('numeric_transformations_knn', numeric_transformations_knn, numeric_cols) # numéricas ahora incluyen MinMax o sea que estan en 0 y 1
    ],
    verbose_feature_names_out=False,   # mantengo nombres de col
    remainder='passthrough'            # dejo pasar lo que no transformo para que no haya problema
).set_output(transform='pandas')        # quiero que salga un DataFrame pq original parece que es array o algo asi

# Aplico el ColumnTransformer actualizado a los datos.
df_minmax = ct_knn_minmax.fit_transform(df_retail)  # ajusto y transformo sobre df_retail
df_minmax = df_minmax[df_retail.columns]            # reordeno columnas como el original porque podrían cambiar el orden

# Exploro antes vs después para ver qué cambió con el escalado.
print("EXPLORACIÓN: df_retail (original)")
explore_data(df_retail, "df_retail")

print("EXPLORACIÓN: df_minmax (IQR + KNN + MinMax)")
explore_data(df_minmax, "df_minmax")

# Resumen numérico para verificar que min≈0 y max≈1 en numéricas.
print("\nResumen rápido numéricos (min/max):")
print("Original (min/max):")
print(df_retail[numeric_cols].agg(['min','max']).T.head())
print("\nMinMax (min/max):")
print(df_minmax[numeric_cols].agg(['min','max']).T.head())

**RESPUESTA VARIACIONES**

Cuando comparamos las distribuciones originales (df_retail) con las transformadas (df_minmax) se notan cambios bien marcados. En el dataset original las variables Price y Quantity tienen escalas gigantes (Price llega a más de 10.000 y Quantity sobre 19.000) y ambas se ven ultra cargadas a la izquierda con colas enormes hacia la derecha, lo que hace que casi toda la masa de datos se aplaste en los primeros bins. Además, había nulos en esas columnas (Quantity ~7.900 y Price ~8.300).

Después de aplicar el pipeline (IQR para cortar outliers, KNN para imputar y MinMax para escalar), las distribuciones cambian harto. Primero, los nulos desaparecen porque ya fueron imputados, y segundo, el rango de valores queda totalmente normalizado entre 0 y 1. Esto se nota en los histogramas de df_minmax, donde Price y Quantity se ven mucho más “extendidas” en el eje x y con colas mucho más controladas. Ya no aparecen esos valores extremos que distorsionaban la escala, porque fueron tratados como outliers antes de escalar.

Entonces la data original mostraba escalas desbalanceadas y con colas largas imposibles de leer, mientras que la versión escalada con MinMax entrega distribuciones más parejas y comparables entre sí, sin nulos y sin distorsión por outliers extremos. Es decir, pasamos de un dataset desordenado y con magnitudes muy distintas a uno más homogéneo y más amigable visualmente.

### 6. Pregunta teórica [0.5 puntos]

<p align="center">
  <img width=300 src="https://file.coinexstatic.com/2023-09-19/166BAC031F222E5910954E7D7D0BC844.png">
</p>

Finalmente, explíquele a Mr. Cheems porqué es útil la creación de pipelines al momento de hacer Feature Engineering en Machine Learning.

**Respuesta:**

Bueno Mr. Cheems, lo que ocurre es que cuando hablamos de *Feature Engineering* nos referimos al proceso de tomar los datos en bruto y transformarlos en variables que sean más claras, útiles y representativas para que un modelo de *Machine Learning* pueda aprender mejor los patrones. La verdad es que no importa tanto si el modelo es complejo o simple, lo que marca la diferencia es la calidad de las variables. O sea, un modelo sencillo, pero con buenas variables puede funcionar mucho mejor que uno complicado con variables mal hechas.

Por lo mismo, aparecen los *pipelines*, que son súper importantes porque permiten ordenar todos los pasos de transformación de los datos antes de entrenar el modelo. En vez de ir haciendo cada cosa a mano una y otra vez, el pipeline asegura que procesos como rellenar valores faltantes, escalar, reducir dimensiones o seleccionar variables se hagan siempre en el mismo orden y de la misma manera. Eso ayuda a evitar errores y da más seguridad en lo que estamos haciendo.

Gracias a los pipelines podemos tener *reproducibilidad* en el código porque cada vez que se corre, los pasos se repiten exactamente igual, sin sorpresas. También ayuda a evitar el *data leakage*, ya que las transformaciones se ajustan solo con los datos de entrenamiento y después se aplican al conjunto de prueba o validación, respetando la separación entre ellos (o sea, los conjuntos no se “conocen” de antes).

Otro punto importante, e igual derivado de lo que explicamos antes, es que los pipelines hacen el trabajo mucho más *rápido y automático* porque basta con llamarlos una vez y todos los pasos se ejecutan sin tener que repetir cada transformación manualmente. Esto además hace que sea más *fácil probar cambios*, porque si quiero reemplazar un paso, lo cambio en el pipeline y se aplica en todo el flujo, sin tener que ir corrigiendo celda por celda.

Y, por último, los pipelines también funcionan muy bien con cosas como la *validación cruzada* y la *búsqueda de hiperparámetros*. Esto básicamente significa que, cuando queremos probar distintas configuraciones de un modelo (por ejemplo, distintos parámetros o combinaciones de transformaciones), el pipeline se encarga de que todo el proceso se ejecute de manera ordenada desde el principio hasta el final. Así, no solo se entrena el modelo, sino que también se aplican siempre las mismas transformaciones a los datos en cada prueba, lo que asegura que el preprocesamiento y el modelado estén realmente conectados y trabajen como un todo.

Entonces, a modo de resumen, los pipelines son súper útiles porque hacen que todo el proceso de preparar los datos y entrenar modelos sea más ordenado, seguro y eficiente. Ayudan a evitar errores, facilitan la experimentación y aseguran que todo el trabajo que hacemos con los datos se mantenga consistente y reproducible, así que es clave usarlo ademas de ser una buena práctica en ciencia de datos.